In [7]:
# 将多个intervention data拼接起来
import copy
import csv
import glob
import numpy as np
import pandas as pd
import os
import random
from cdt.metrics import get_CPDAG

def add_lag(X, lag):
    if lag == 0: return X
    for i in range(1,lag+1):
        X_i = np.repeat(X[0,:], axis=0, repeats=lag)  
        X_i = np.vstack((X_i,X[lag:,:]))
        X_all = np.hstack((X,X_i))
    return X_all

def generate_whole(load_path, save_path, i_dataset, interv_list, normal_num, num_lags):
    print(interv_list, normal_num)
    time_points = 120
    num_interv = len(interv_list)
    for i in range(num_interv+1): 
        if i==0: # normal
            normal_path = f'/normal/{normal_num}_normal.npy'
            i_normal = np.load(load_path+normal_path)
            # ############
            # df = pd.DataFrame(i_interv)
            # df = df.apply(lambda x : (x-np.min(x))/(np.max(x)-np.min(x)))
            # i_interv = df.to_numpy()
            # ##########
            i_normal_lag = add_lag(i_normal, num_lags)
            i_regime = np.repeat(i, axis=0, repeats=time_points)  
            data_full = copy.deepcopy(i_normal_lag)
            # print(data_interv.shape)
            np.save(save_path+f'data{i_dataset}.npy', data_full)
            regime = copy.deepcopy(i_regime)  
            with open(save_path+f'intervention{i_dataset}.csv','w',newline='') as intervention:
                writer = csv.writer(intervention)
                for _ in range(time_points):
                    writer.writerow([])
            data_interv = data_full[:time_points,:]
                    
        else:    # interv, concat data_interv, regime
            i_interv = np.load(load_path+f'abnormal/{interv_list[i-1]}_interv.npy')
            # ############
            # df = pd.DataFrame(i_interv)
            # df = df.apply(lambda x : (x-np.min(x))/(np.max(x)-np.min(x)))
            # i_interv = df.to_numpy()
            # ##########
            i_interv_lag = add_lag(i_interv, num_lags)
            i_regime = np.repeat(i, axis=0, repeats=i_interv.shape[0])
            data_interv = np.concatenate((data_interv,i_interv_lag),axis=0)
            regime = np.concatenate((regime,i_regime),axis=0)
            # concat i_label 
            i_label = open(load_path+f'/abnormal/{interv_list[i-1]}_label.csv','r').read()
            with open(save_path+f'intervention{i_dataset}.csv','a') as intervention:
                intervention.write(i_label)
                
    print(data_interv.shape, regime.shape)
    np.save(save_path+f'data_interv{i_dataset}.npy', data_interv)
    np.savetxt(save_path+f'regime{i_dataset}.csv', regime, fmt='%d',delimiter=None)

def generate_empty_DAG(num_nodes, num_lags, save_path, i_dataset):
    # dag = np.ones((num_nodes*(1+num_lags),num_nodes*(1+num_lags)))
    dag = np.random.randint(0,1,(num_nodes*(1+num_lags),num_nodes*(1+num_lags)))
    cpdag = get_CPDAG(dag)
    print(dag.shape)
    np.save(save_path+f'DAG{i_dataset}.npy', dag)
    np.save(save_path+f'CPDAG{i_dataset}.npy', cpdag)


In [8]:
num_raw = 9 # 12
num_normal = 7
num_gen_datasets = 7
num_interv_each = 5
num_nodes = 36 # 38
num_lags = 1
load_path = '/home/lipeiwen.lpw/realdata_preprocess/transfered/upstream_less3/' # upstream
save_path = '/home/lipeiwen.lpw/realdata_preprocess/train_data/upstream_less3/' # upstream
save_path_train = '/home/lipeiwen.lpw/temporal_intervention/data_real/new/upstream_less3/'
os.makedirs(save_path,exist_ok=True)
# 任选 normal data
normal_num = random.sample(range(1,num_normal+1),num_gen_datasets)
# 任选 abnormal data
abnormal_lists = set()
while len(abnormal_lists)<num_gen_datasets:
    i_numbers = random.sample(range(1,num_raw+1),num_interv_each)
    abnormal_lists.add(tuple(i_numbers))
abnormal_lists = sorted(abnormal_lists)

for i in range(num_gen_datasets):
    i_dataset = i+1
    generate_whole(load_path = load_path, 
                   save_path = save_path,
                   i_dataset = i_dataset,
                   interv_list = abnormal_lists[i],
                   normal_num = normal_num[i],
                   num_lags = num_lags)
    generate_empty_DAG(num_nodes = num_nodes, 
                       num_lags = num_lags, 
                       save_path = save_path, 
                       i_dataset = i_dataset)

(1, 2, 7, 4, 5) 7
(720, 72) (720,)
(72, 72)
(1, 8, 2, 5, 7) 4
(720, 72) (720,)
(72, 72)
(3, 9, 1, 7, 6) 1
(720, 72) (720,)
(72, 72)
(5, 4, 2, 7, 1) 2
(720, 72) (720,)
(72, 72)
(5, 4, 9, 6, 3) 5
(720, 72) (720,)
(72, 72)
(9, 3, 2, 4, 6) 3
(720, 72) (720,)
(72, 72)
(9, 8, 1, 3, 2) 6
(720, 72) (720,)
(72, 72)


In [9]:
import shutil
shutil.copytree(save_path,save_path_train,dirs_exist_ok=True)

'/home/lipeiwen.lpw/temporal_intervention/data_real/upstream_less3/'

In [10]:
# load_path = '/home/lipeiwen.lpw/realdata_preprocess/train_data/downstream/' # upstream
# i_dataset = 2
# df = pd.DataFrame(np.load(load_path+f'data_interv{i_dataset}.npy'))
# df.var()

In [11]:
# generate_empty_DAG(num_nodes = 20, 
#                        num_lags = 1, 
#                        save_path = '/home/lipeiwen.lpw/temporal_intervention/data_timesery/imperfect/20_p1_smp600/', 
#                        i_dataset = 1)